# Instalação

Para instalar o LangChain, execute:


In [1]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\felip\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# LangSmith

Muitas das aplicações que você constrói com LangChain contêm múltiplas etapas com várias chamadas de modelos de linguagem (LLMs). À medida que essas aplicações se tornam mais complexas, torna-se crucial poder inspecionar exatamente o que está acontecendo dentro da sua cadeia ou agente. A melhor maneira de fazer isso é com o LangSmith.

Após se cadastrar no link acima, certifique-se de definir suas variáveis de ambiente para começar a registrar rastreamentos:  


In [2]:
#export LANGSMITH_TRACING="true"
#export LANGSMITH_API_KEY="..."
#export LANGSMITH_PROJECT="default" # or any other project name

Ou, se estiver em um notebook, você pode defini-las com:


In [3]:
import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"

# Usando Modelos de Linguagem

Primeiramente, vamos aprender como usar um modelo de linguagem isoladamente. O LangChain oferece suporte a vários modelos de linguagem diferentes que podem ser usados de forma intercambiável.


In [4]:
pip install -qU "langchain[google-genai]"

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.41.1 requires protobuf<6,>=3.20, but you have protobuf 6.31.1 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\felip\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

Vamos primeiro usar o modelo diretamente. Os ChatModels são instâncias de *Runnables* do LangChain, o que significa que eles expõem uma interface padrão para interagir com eles. Para simplesmente chamar o modelo, podemos passar uma lista de mensagens para o método `.invoke`.


In [6]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

model.invoke(messages)

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


AIMessage(content='Ciao!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--9baf4b94-63c0-40c5-a479-45364ff0acae-0', usage_metadata={'input_tokens': 9, 'output_tokens': 3, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTP

Observe que os ChatModels recebem objetos de mensagem como entrada e geram objetos de mensagem como saída. Além do conteúdo em texto, os objetos de mensagem indicam papéis conversacionais e contêm dados importantes, como chamadas de ferramentas e contagem de tokens utilizados.

O LangChain também oferece suporte a entradas para modelos de chat via strings ou no formato da OpenAI. As seguintes formas são equivalentes:


In [7]:
model.invoke("Hello")

model.invoke([{"role": "user", "content": "Hello"}])

model.invoke([HumanMessage("Hello")])

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--ba3e1343-2880-4518-86d5-6f6849f2626b-0', usage_metadata={'input_tokens': 1, 'output_tokens': 10, 'total_tokens': 11, 'input_token_details': {'cache_read': 0}})

# Transmissão (Streaming)

Como os modelos de chat são *Runnables*, eles expõem uma interface padrão que inclui modos de invocação assíncronos e por transmissão (*streaming*). Isso nos permite transmitir tokens individuais de um modelo de chat:


In [8]:
for token in model.stream(messages):
    print(token.content, end="|")

Ciao|!
|

Os *prompt templates* são um conceito do LangChain criado para ajudar nessa transformação. Eles recebem uma entrada bruta do usuário e retornam dados (um prompt) prontos para serem passados a um modelo de linguagem.

Vamos criar um *prompt template* aqui. Ele aceitará duas variáveis do usuário:

- `language`: O idioma para o qual o texto será traduzido  
- `text`: O texto a ser traduzido


In [9]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

Observe que o `ChatPromptTemplate` suporta múltiplos papéis de mensagem em um único template. Formatamos o parâmetro `language` na mensagem do sistema, e o texto do usuário em uma mensagem do usuário.

A entrada para esse *prompt template* é um dicionário. Podemos experimentar esse *prompt template* isoladamente para ver o que ele faz por si só.


In [10]:
prompt = prompt_template.invoke({"language": "Italian", "text": "hi!"})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

In [11]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]

In [12]:
response = model.invoke(prompt)
print(response.content)

Ciao!
